In [14]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
from sklearn import grid_search

Populating the interactive namespace from numpy and matplotlib


In [2]:
from IPython.parallel import Client
rc = Client()

In [15]:
alexnet_train = np.load("CS5785-final-data/alexnet_feat_train.npy")
sift_train = np.load("CS5785-final-data/SIFTBoW_train.npy")
attribute = np.genfromtxt("CS5785-final-data/attributes_train.txt",dtype='str')[:,1]
train = np.genfromtxt("CS5785-final-data/train.txt",dtype='str')
alexnet_test = np.load("CS5785-final-data/alexnet_feat_test.npy")
test_id = np.genfromtxt("CS5785-final-data/test.txt",dtype='str')
train_label = list(train[:,1])

In [4]:
from sklearn.preprocessing import scale
scale_alexnet = scale(alexnet_train, axis=0, with_mean=True, with_std=True, copy=True)
scale_alexnet_test = scale(alexnet_test, axis=0, with_mean=True, with_std=True, copy=True)

[[-0.7798487  -1.68445003 -1.19879558 ..., -0.63580746 -0.40296847
   0.5512287 ]
 [-0.71109723  0.05550532  0.68368766 ...,  0.63552846 -0.56516745
   0.82728138]
 [ 0.02707332 -0.1759728   1.71622296 ..., -0.31332588  1.28494866
   0.09507528]
 ..., 
 [-0.87849943  0.98975693  0.32895484 ...,  2.00567114  0.15881243
   1.11710018]
 [-1.01087352  0.63344114  1.42269969 ..., -0.33161848  1.48116542
  -0.32136853]
 [ 0.0399384  -1.02224589 -0.77870236 ..., -0.61645178  1.30974869
   0.67064072]]


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import cross_validation

In [6]:
svn = SVC()
param_grid = [
  {'C': [0.01, 1, 10, 100, 1000, 10000], 'kernel': ['linear']},
  {'C': [0.01, 1, 10, 100, 1000, 10000], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
 ]
clf = grid_search.GridSearchCV(svn, param_grid)
clf.fit(scale_alexnet,train_label)
print clf.grid_scores_

[mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 0.01}, mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 1}, mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 10}, mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 100}, mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 1000}, mean: 0.37800, std: 0.00589, params: {'kernel': 'linear', 'C': 10000}, mean: 0.24233, std: 0.00946, params: {'kernel': 'rbf', 'C': 0.01, 'gamma': 0.001}, mean: 0.34733, std: 0.00680, params: {'kernel': 'rbf', 'C': 0.01, 'gamma': 0.0001}, mean: 0.31700, std: 0.00589, params: {'kernel': 'rbf', 'C': 0.01, 'gamma': 1e-05}, mean: 0.30267, std: 0.00736, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}, mean: 0.36867, std: 0.00655, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}, mean: 0.31700, std: 0.00589, params: {'kernel': 'rbf', 'C': 1, 'gamma': 1e-05}, mean: 0.30467, std: 0.00544, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}, mean: 0.3933

In [7]:
print clf.best_params_,clf.best_score_

{'kernel': 'rbf', 'C': 10, 'gamma': 0.0001} 0.393333333333


In [12]:
predicts = clf.predict(scale_alexnet_test)

In [19]:
import csv
with open("svn_result",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["ID","Category"])
    for i in range(len(predicts)):
        writer.writerow([test_id[i],predicts[i]])

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
scale_alexnet = MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(scale_alexnet)
normal_alexnet = normalize(scale_alexnet, norm='l2', axis=1, copy=True)
#print normal_alexnet
mnb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
score = cross_validation.cross_val_score(mnb,normal_alexnet,train_label,cv=5)

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
gnb = GaussianNB()
#print alexnet_train
#clf.fit(alexnet_train,train_label)
score = cross_validation.cross_val_score(gnb,scale_alexnet,train_label,cv=5)

In [8]:
print score

[ 0.37333333  0.38166667  0.37666667  0.37833333  0.36666667]


In [9]:
svn = SVC(kernel='rbf',C=10,gamma=0.0001)

In [10]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, bootstrap=False, oob_score=False, random_state=None, verbose=0,class_weight=None)

In [24]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation 
bnb = BernoulliNB(alpha=0.14, binarize=0.8, fit_prior=True, class_prior=None)
svn = SVC(kernel='linear',C=1.0,gamma=0.001,decision_function_shape='ovr')
logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.005, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='ovr', verbose=0)

In [25]:
from sklearn.preprocessing import scale
scale_alexnet = scale(alexnet_train, axis=0, with_mean=True, with_std=True, copy=True)
scale_alexnet_test = scale(alexnet_test, axis=0, with_mean=True, with_std=True, copy=True)

In [26]:
from sklearn.ensemble import VotingClassifier
voteclf = VotingClassifier(estimators=[('lr', logreg), ('bnb',bnb), ('svn', svn)],voting='hard')
score = cross_validation.cross_val_score(voteclf,scale_alexnet,train_label,cv=3)
print score

[ 0.396  0.419  0.419]


In [27]:
voteclf.fit(scale_alexnet,train_label)
vote_predicts = voteclf.predict(scale_alexnet_test)

In [28]:
import csv
with open("majorvote_result",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["ID","Category"])
    for i in range(len(vote_predicts)):
        writer.writerow([test_id[i],vote_predicts[i]])

In [24]:
print score

[ 0.394  0.403  0.4  ]


In [27]:
from sklearn.ensemble import voting_classifier